### Functions

In [9]:
from itertools import product, combinations_with_replacement, combinations
import numpy as np
from scipy import stats
from typing import List, Callable
from math import sqrt

In [ ]:
def prob(k, j, fdr=alpha,m=M,n=N,delta=delt,variance=var,size=s, null=null_test):
    #edge case: k=0
    assert m>n
    return sum([expand(k, j, i, m, n, fdr, null, delta,variance,size) for i in range(1, m - k+2)])


def expand(k, j, i, m, n, fdr, null, delta,variance,size):
    # find constrained permutation
    # combinations = sorted permutations only
    permutation = combinations(np.arange(k , m+1),i)
    # k is fixed, not iterating
    permutation = list(filter(lambda x: list(x)[0]==k, permutation))
    
    total = 0
    
    for perm in permutation:
        assert f_ordinal(perm, k, j, m, n, fdr, null, delta,variance,size)>=0 and f_ordinal(
            perm, k, j, m, n, fdr, null, delta,variance,size)<1, f'invalid when perm={perm}, k={k},j={j}'
                
        total += f_ordinal(perm, k, j, m, n, fdr, null, delta,variance,size)
    
    #when i is odd, *1 otherwise *(-1)
    return (-1) ** (i+1) * total


# p-value's cdf     
def F(i,b,null,fdr,delta,variance,m,n,size):
    #temporary set it to be 0 when uncertain about F_0 
    if i==0:
        return 0
    
    #uniform when it's a true null 
    if i in null:
        F=b

    else:
        F=stats.norm.cdf(stats.norm.ppf(b/2)-delta/(variance/np.sqrt(size)))+1-stats.norm.cdf(
            stats.norm.ppf(1-b/2)-delta/(variance/np.sqrt(size)))
    return F


# BH rejection bounds
def b(perm,fdr,m):
    # b_n0=0, b_n_e+1=1, b_n1...b_n_e= BH rejection bounds for n_1,...,n_e; e=len(perm)
    b0_e=np.append(0,np.array(perm))*fdr/m
    return np.append(b0_e,1)

# I index set, =[i_vec_0,...,i_vec_e+1]
def I(perm, m):
    I_vec = combinations_with_replacement(range(m+1), len(perm) + 2)
    # i_vec_0=0, i_vec_e+1=m
    I_vec = filter(lambda x: x[0]==0 and x[len(perm)+1]==m, I_vec)
    # non-descending order
    I_vec = filter(lambda x: sorted(x) == list(x), I_vec)
    # i_h>=n_h for h=1,...,len(perm)
    I_vec=filter(lambda x: all(i >= perm[y] for y,i in enumerate(x[1:len(perm)+1])),I_vec)
    return list(I_vec)



# M index set; μ's size=[m,e+1]
# i_vec is a vector in I(perm)
def Ms(perm, i_vec, n_row, true_null, false_rejections):
    assert n_row >= true_null
    assert false_rejections <= true_null
    n_col = len(perm)+1
    
    # constraint 11
    possible_rows = np.identity(n_col)
    candidates = [*product(possible_rows, repeat=n_row)] 
    
    final_M = []
    for arr in candidates:
        # constaint 9
        if np.sum(np.array(arr)[0:true_null, 0]) == false_rejections: 
            # constraint 10
            if np.all(np.sum(np.array(arr)[::, 1:],axis=0) == [(i_vec[h+1] - i_vec[h]) for h in range(1, n_col)]): 
                final_M.append(arr)
                
    return final_M


# formula 12: find constrained permutation
def f_ordinal(perm, k, j, m, n, fdr, null, delta,variance,size):
    # I summation
    I_set = I(perm=perm, m=m)
    total = 0
    
    for i_vec in I_set:
        M_set = Ms(perm,i_vec, n_row=m,true_null=n, false_rejections=j)
        if M_set!=[]:
            for μ in M_set:
                for h in range(len(perm)+1):
                    qualified_i=np.nonzero(np.array(μ)[:,h])[0]+1# find i s.t. μ=1
                    if len(qualified_i)==0:
                        continue
                    else:
                        subtotal=1
                        for i in qualified_i:
                            
                            subtotal *= F(i, b(perm=perm, fdr=fdr,m=m)[h + 1],null=null,fdr=fdr,delta=delta,variance=
                                          variance,m=m,n=n,size=size) - F(
                                i, b(perm=perm, fdr=fdr,m=m)[h],null=null,fdr=fdr,delta=delta,variance=
                                          variance,m=m,n=n,size=size)
                        total+=subtotal
    return total

### Calculations

In [52]:
# parameters from table 1
delt = 1
var = 1
alpha = 0.05
s = 5
M = 5
N = 3
null_test=[1,2,3]

In [ ]:
K = np.arange(1, M + 1)
J = np.arange(N)
false_null = M - N
avg_pow = 0

# formula(14)
for [k, j] in np.array(np.meshgrid(K, J)).T.reshape(-1, 2):
    if k <= j:
        continue
    print(k,j,(k - j) / false_null * prob(k, j,fdr=alpha,m=M,n=N,delta=delt,variance=var,size=s, null=null_test))
    avg_pow += (k - j) / false_null * prob(k, j, fdr=alpha,m=M,n=N,delta=delt,variance=var,size=s, null=null_test)

print(avg_pow)

### Bug

In [53]:
prob(k=1, j=1, fdr=alpha,m=M,n=N,delta=delt,variance=var,size=s, null=null_test)

AssertionError: invalid when perm=(1,), k=1,j=1

In [56]:
f_ordinal((1,), k, j, M, N, alpha, null_test, delt,var,s)

3.970299

In [58]:
F(1, b(perm=(1,), fdr=alpha,m=M)[2],null=null_test,fdr=alpha,delta=delt,variance=
                                          var,m=M,n=N,size=s) - F(
                                1, b(perm=(1,), fdr=alpha,m=M)[1],null=null_test,fdr=alpha,delta=delt,variance=
                                          var,m=M,n=N,size=s)

0.99

Note: since b[e+1]=1, F(i,b[e+1])=prob(ith hypothesis' p-value<=1)=1 for all i. b[e] is usually very small since it's rejection boundary under BH procedure. This makes all F(i,b[e+1])-F(i,b[e]) very close to 1 thus the results of formula 12(function f_ordinal) is always larger than one for all i and all perm.